In [43]:
import openai
import pandas as pd
import requests
import os
from io import BytesIO
from openai import OpenAI
import concurrent.futures
import base64
import time

In [44]:
# OpenAI API Key
api_key = ""

In [49]:

# Specify the directory containing the images
image_dir = "output_folder"

# DataFrame to store the results
results_df = pd.DataFrame(columns=['image_id', 'vision_openai_sp_detection', 'vision_openai_response_time'])

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_image_description(base64_image, api_key):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Which Landmark is in the image? Indicate only one that matches one of the names of the landmarks from this list: Tower Bridge, The British Museum, Big Ben, Lastminute.com London Eye, Tower Of London, Trafalgar Square, Westminster Bridge, 30 St Mary Axe (The Gherkin), Palace Of Westminster, The Shard, Wembley Stadium, None"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens":  20
    }
    start_time = time.time()  # Start timing
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    end_time = time.time()
    vision_openai_response_time = end_time - start_time
    if response.status_code == 200:
        vision_openai_sp_detection = response.json()['choices'][0]['message']['content']
        return vision_openai_sp_detection, vision_openai_response_time
    else:
        return "Error in API call", vision_openai_response_time




In [50]:
#Loop through all images in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):  # Adjust this if your images have a different format
        image_id = filename.split('.')[0]
        image_path = os.path.join(image_dir, filename)
        
        # Encode the image
        base64_image = encode_image(image_path)
        
        # Get the image description from the Vision API
        vision_openai_sp_detection, vision_openai_response_time = get_image_description(base64_image, api_key)  # Replace with your actual API key
        
        # Store the results in the DataFrame
        results_df = results_df.append({
            "image_id": image_id,
            "vision_openai_sp_detection": vision_openai_sp_detection,
            "vision_openai_response_time": vision_openai_response_time
        }, ignore_index=True)



/var/tmp/ipykernel_2619020/3290182863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/tmp/ipykernel_2619020/3290182863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/tmp/ipykernel_2619020/3290182863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/tmp/ipykernel_2619020/3290182863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/var/tmp/ipykernel_2619020/3290182863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [41]:
### from this list: Tower Bridge, The British Museum, Big Ben, Lastminute.com London Eye, Tower Of London, Trafalgar Square, Westminster Bridge, 30 St Mary Axe (The Gherkin), Palace Of Westminster, The Shard, Wembley Stadium, None"
     

In [52]:
# Save the results to a CSV file
results_df

,image_id,vision_openai_sp_detection,vision_openai_response_time
0,AQ7Q666WK2OXZKIQBQVWPRC5D2M6XFBEHFKQ3MRFV2T2OM...,Big Ben,6.470501
1,TKUAUGKMUUXUYKFE53MLJ64UC2LBDZK22CKY2JXF22ELUS...,None,3.266893
2,5FF4Y4LHR5EYDJ7K25U5HUMKWANKYONMVSHQ7ILHOHAX53...,The Shard,13.263211
3,OQNB4FXPW3SUEF2CRCJNENGESSWUNL5XPYR2BTGLYQ7MRG...,The landmark in the image is Tower Bridge.,2.948924
4,7LZ6QBXCSKYIUJUSJK2OO3VCHQ5CTWVZVQRGB3C6K6KUR4...,The landmark in the image is the National Gall...,2.123317
...,...,...,...
266,WTTKN72SLSR7MMY5TVF6ITSI2PT3WZTYFY6PZPIO7KQ2DL...,Error in API call,0.617543
267,KQAGOP6B4V4GKGXQNDDLQDRY2PLQ35LGHAHGXGVEEI5OKJ...,The British Museum,2.079219
268,AKM4JAMFM7CCHTFK72ACCQMHZ5LF6POZZPB2JANI7DY4Y2...,Tower Bridge,5.316881
269,DPCW35BXU2KSMYUIGNDWCMADZZEAQMP67PN7FUCX3EHZAL...,Lastminute.com London Eye,10.323751


In [51]:
results_df.to_csv("openai_vision_sp_detection.csv", index=False)

In [ ]:
df